In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Screenshot-2024-10-01-at-1.49.21 PM.jpg to Screenshot-2024-10-01-at-1.49.21 PM.jpg
Saving Telecom.jpg to Telecom.jpg


In [ ]:
from PIL import Image
image = [Image.open('/content/1727056685.jpg'), Image.open('/content/Screenshot-2024-10-01-at-1.49.21 PM.jpg'), Image.open('/content/Telecom.jpg')]

**Google Gemini 1.5 Flash**

In [ ]:
# @title
from PIL import Image
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('Google_API_Key')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# @title
# for model in genai.list_models():
#   print(model)

In [ ]:
# @title
def get_gemini_response(input,image):
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    response = model.generate_content([input,image])
    return response.text

In [ ]:
# @title
input = ''' You are an expert in understanding Invoices. Now answer the following questions from the Invoice given
        1.What is the Invoice Number ?
        2.What is the Invoice Date ?
        3.What is the Invoice Total ?
        4.What is the Customer Name ?
        '''

for i,img in enumerate(image,start=1):

  print(f'\n\nExtracted Info for Image_{i} \n')
  response = get_gemini_response(input,img)
  entities = {}

  # Iterate through each line of the response
  for line in response.split('\n'):
    if ':**' in line:  # Match the key-value separator
        # Remove sequential number and dot using split
        line = line.split('. ', 1)[-1]
        key, value = line.split(':**', 1)
        key = key.replace('**', '').strip()  # Remove '**' and trim whitespace
        entities[key] = value.strip()  # Trim whitespace for values

  print(entities)




Extracted Info for Image_1 

{'Invoice Number': '22PA6LLJ00005001', 'Invoice Date': '09/09/2022', 'Invoice Total': '124.96', 'Customer Name': 'Akanksha'}


Extracted Info for Image_2 

{'Invoice Number': '24WCHGBB00002615', 'Invoice Date': '30/06/2024', 'Invoice Total': '358.262', 'Customer Name': 'Sarang S'}


Extracted Info for Image_3 

{'Invoice Number': 'GST-6290', 'Invoice Date': '26-01-2020', 'Invoice Total': '69,990.00  (Rupees Sixty Nine Thousand Nine Hundred Ninty Only)', 'Customer Name': 'IDFC FIRST BANK LTD (FINANCE)'}


**Phi3 Vision on GPU**

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 14.4 MB/s eta 0:00:00


In [ ]:
!pip install pillow==10.3.0 chardet==5.2.0 flash_attn==2.5.8 accelerate==0.30.1 bitsandbytes==0.43.1 Requests==2.31.0 transformers==4.40.2 albumentations==1.3.1 opencv-contrib-python==4.10.0.84 matplotlib==3.9.0

  Using cached pillow-10.3.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
  Using cached flash_attn-2.5.8.tar.gz (2.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached accelerate-0.30.1-py3-none-any.whl.metadata (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 9.8 MB/s eta 0:00:00
  Using cached ninja-1.11.1.2-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.3 kB)
Using cached pillow-10.3.0-cp310-cp310-manylinux_2_28_x86_64.whl (4.5 MB)
Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3

In [ ]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor
from transformers import BitsAndBytesConfig
import torch
from IPython.display import display
import time

In [ ]:
# Define model ID
model_id = "microsoft/Phi-3-vision-128k-instruct"

# Load processor
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

# Define BitsAndBytes configuration for 4-bit quantization
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load model with 4-bit quantization and map to CUDA
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    trust_remote_code=True,
    torch_dtype="auto",
    quantization_config=nf4_config,
)

config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

configuration_phi3_v.py:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- configuration_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3_v.py:   0%|          | 0.00/75.4k [00:00<?, ?B/s]

image_embedding_phi3_v.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- image_embedding_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- modeling_phi3_v.py
- image_embedding_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
def model_inference(messages, path_image):

    start_time = time.time()

    image = Image.open(path_image)

    # Prepare prompt with image token
    prompt = processor.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # Process prompt and image for model input
    inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")

    # Generate text response using model
    generate_ids = model.generate(
        **inputs,
        eos_token_id=processor.tokenizer.eos_token_id,
        max_new_tokens=500,
        do_sample=False,
    )

    # Remove input tokens from generated response
    generate_ids = generate_ids[:, inputs["input_ids"].shape[1] :]

    # Decode generated IDs to text
    response = processor.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]


    display(image)
    end_time = time.time()
    print("Inference time: {}".format(end_time - start_time))

    # Print the generated response
    print(response)

In [ ]:
prompt_cie_front = [{"role": "user", "content": "<|image_1|>\nOCR the text of the image. Extract the text of the following fields and put it in a JSON format: \
'Comune Di/ Municipality', 'COGNOME /Surname', 'NOME/NAME', 'LUOGO E DATA DI NASCITA/\
PLACE AND DATE OF BIRTH', 'SESSO/SEX', 'STATURA/HEIGHT', 'CITADINANZA/NATIONALITY',\
'EMISSIONE/ ISSUING', 'SCADENZA /EXPIRY'. Read the code at the top right and put it in the JSON field 'CODE'"}]

# Download image from URL
path_image = "/home/randellini/llm_images/resources/cie_fronte.jpg"

# inference
model_inference(prompt_cie_front, path_image)

**Ollama Vision Model LLama3.2Vision**

In [ ]:
# !pip install colab-xterm ollama

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh



import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 2s (226 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
55 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as

In [ ]:
# !ollama serve
!ollama pull llama3.2-vision

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 11f274007f09... 100% ▕▏ 6.0 GB                         
pulling ece5e659647a... 100% ▕▏ 1.9 GB                         pulling manifest 
pulling 11f274007f09... 100% ▕▏ 6.0 GB                         
pulling ece5e659647a... 100% ▕▏ 1.9 GB                         
pulling 715415638c9c... 100% ▕▏  269 B                         
pulling 0b4284c1f870... 100% ▕▏ 7.7 KB                         
pulling fefc914e46e6... 100% ▕▏   32 B                         
pulling fbd313562bb7... 100% ▕▏  572 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
!ollama list

NAME                      ID              SIZE      MODIFIED               
llama3.2-vision:latest    38107a0cd119    7.9 GB    Less than a second ago    


In [ ]:
image = ['/content/1727056685.jpg','/content/Screenshot-2024-10-01-at-1.49.21 PM.jpg','/content/Telecom.jpg']

In [ ]:
import ollama
for i,img in enumerate(image,start=1):
  print(f'\n\nExtracted Info for Image_{i} \n')
  response = ollama.chat(
      model='llama3.2-vision',
      messages=[{
          'role': 'user',
          'content': ( '''
              "You are an expert in understanding and analyzing invoices. "
              "Now, Just answer the following queries based on the provided invoice image in python dictionary format, nothing more that that"
              1. Invoice Number, 2.Invoice Date, 3.Customer Name, 4.Total Amount
               The response format should be in the following format:

               Sure, here is the response in python dictionary format:

               {
                "Invoice Number": "22PA6LLJ00005001",
                "Invoice Date": "09/09/2022",
                "Customer Name": "Akanksha",
                "Total Amount": 124.96
               }
          '''),
          'images': [img]
      }]
  )
  entities={}
  for line in response.message['content'].split('\n\n')[1].replace('"',' ').split('\n'):
        if ':' in line:
          key, value = line.split(':', 1)
          entities[key.strip()] = value.strip()
  print(entities)



Extracted Info for Image_1 

{'Invoice Number': '22PA6LLJ00005001 ,', 'Invoice Date': '09/09/2022 ,', 'Customer Name': 'Akanksha ,', 'Total Amount': '124.96'}


Extracted Info for Image_2 

{'Invoice Number': '24WCHGBB00002615 ,', 'Invoice Date': '30/06/2024 ,', 'Customer Name': 'Sarang S ,', 'Total Amount': '358.262'}


Extracted Info for Image_3 

{'Invoice Number': 'GST-6290 ,', 'Invoice Date': '26-01-2020 ,', 'Customer Name': 'IDFC FIRST BANK LTD (FINANCE) ,', 'Total Amount': '69,990.00'}
